# Test bot

In [1]:
from ollama_models import ollama_models
models = ollama_models()

import os
for model in models:
    dn = f'forecast_{model}'
    os.makedirs(dn, exist_ok=True)
    print("created", dn)

created forecast_cogito:latest
created forecast_deepseek-r1:latest
created forecast_gemma3:latest
created forecast_granite3.3:latest
created forecast_llama3:8b
created forecast_llama4:latest
created forecast_magistral:24b
created forecast_mistral-small3.1:latest
created forecast_openthinker:latest
created forecast_phi4-reasoning:latest
created forecast_qwen2.5vl:latest
created forecast_qwen3:latest
created forecast_r1-1776:latest


In [2]:
hard_ids = [6614, 1482, 1337, 7811, 8734, 5531, 1454, 6633, 2622, 3054]

from load_saved_questions import load_saved_questions
questions = load_saved_questions(hard_ids)

from community_forecast import community_forecast
id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}

import pandas as pd
from flatten_dict import flatten_dict
from datetime import datetime
df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
df['id_of_post'] = [q.id_of_post for q in questions]
df['id_of_question'] = [q.id_of_question for q in questions]
df['question_options'] = df['question_options'].apply(repr)
df['today'] = datetime.now().strftime("%Y-%m-%d")
df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)
df = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title', 'question_description',
    'question_resolution_criteria', 'question_fine_print', 'question_type',
    'question_options', 'question_group_variable', 'question_question_weight',
    'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
    'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']]

from gather_research_and_set_prompt import gather_research_and_set_prompt
live=True
df, rag = gather_research_and_set_prompt(df, live)

2025-06-18 16:42:18.526 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Loaded existing index from forecast_index.faiss
Index contains 4777 vectors at initialization


In [3]:
from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
from tqdm import tqdm
for model in tqdm(models):
    print("Model", model)
    try:
        df = generate_forecasts_and_update_rag(df, rag, live, model)
        print(df)
    except:
        print("Model run failed", model)

  8%|███▍                                        | 1/13 [00:00<00:01,  6.85it/s]

Model cogito:latest
=== Starting Forecast ===
already done forecast_cogito:latest/6614.md
already done forecast_cogito:latest/1482.md
already done forecast_cogito:latest/1337.md
already done forecast_cogito:latest/7811.md
already done forecast_cogito:latest/8734.md
already done forecast_cogito:latest/5531.md
already done forecast_cogito:latest/1454.md
already done forecast_cogito:latest/6633.md
already done forecast_cogito:latest/2622.md
already done forecast_cogito:latest/3054.md
Atomically saved RAG state
   id_of_question  id_of_post       today             open_time  \
0            6614        6614  2025-06-18  2021-03-15T23:00:00Z   
1            1482        1482  2025-06-18  2018-10-06T07:00:00Z   
2            1337        1337  2025-06-18  2018-08-11T07:00:00Z   
3            7811        7811  2025-06-18  2021-08-23T13:00:00Z   
4            8734        8734  2025-06-18  2021-11-26T08:00:00Z   
5            5531        5531  2025-06-18  2020-10-29T07:00:00Z   
6            1454 


  8%|███▍                                        | 1/13 [00:13<02:40, 13.33s/it]


KeyboardInterrupt: 